<a href="https://colab.research.google.com/github/kylevirtuous1211/Multi-Agent-Simulation-on-Customer-Behavior/blob/main/AI_Agent_CoT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import userdata

In [2]:
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key

In [3]:
!pip install aisuite[all]
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.11.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━

## Chain of Thought
1. plan: list some subtasks
2. solve subtasks with LLM and return results
3. Give the reasoning model

In [4]:
import aisuite as ai

In [5]:
def reply (prompt="Hi", system="You're a unbiased customer that gives feedback on the product that hasn't be on the internet",
           provider="groq",
           model="llama3-70b-8192"):
  client = ai.Client()
  messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": prompt}
  ]
  response = client.chat.completions.create(
      messages=messages,
      model=f"{provider}:{model}"
  )

  return response.choices[0].message.content

## make a 2 stage planner

In [6]:
system_planner = "List some (more than one) customer persona would buy the product from the description of the prompt"
system_writer = "According to one of the persona, write a review of how customer would react."
system_critic = "based on how it reacts, return how the product are expected to be better or some critical points"
system_verdict = "according to the reaction and critical points, give a verdict of whether would buy or not"

In [7]:
def customer_behavior(prompt = "product design, cost, texture, appearance..."):
  planning_prompt = f"This is the product that we are testing the customer reaction: {prompt}"
  found_persona = reply(prompt=planning_prompt, system=system_planner)

  writer_prompt = f"write the review for {prompt}, based on {found_persona}"
  review = reply(prompt=writer_prompt, system=system_writer)

  critic_prompt = f"from the description: {prompt}, give some potential critical points of the product"
  critics = reply(prompt=critic_prompt, system=system_critic)

  system_verdict = f"based on {review}, {critics}. Give a final verdict out of 100 of the probability of customer buying intention"
  verdict = reply(prompt=system_verdict, system=system_verdict)

  return found_persona, review, critics, verdict

# 結果

## 實驗設計
這禮拜的目標與以前相同，想要用ai agent預測市場客群的行為，我運用了4個Chain of Thought的agent來進行這次的模擬。
1. CoT1: 先列出可能購買的目標客群
2. CoT2: 針對對產品的第一印象進行描寫 (LLM回答幾乎都是正面的)
3. CoT3: 列出可能的負面點
4. 最終review: 針對正向、負向的市場觀點，進行最後市場客群行為結果的撰寫。

## 實驗結果：
我發現用多個LLM更能減少最終回答的hallucination，將原本工作divide and conquer，分別交給不同的LLM，就能忠實地達到想要的目標
### 優點
* 成功最後給出量化購買意願 (out of 100)
* 指令都有據實的達到，最後總結我也蠻滿意的
### 缺點
* 回答有點太發散，且太多情緒用詞 => 可能不能只用敘述 (給模型的資訊太少了)
* 缺乏真實數據支持 => 需要RAG
* 缺乏real time input => 可以加上網路搜尋工具


In [8]:
# prompt: use gr.Row() to generate outputs in different textbox

import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# Simulation of Customer Behavior")
    gr.Markdown("## enter a description of your product: we can analyze customer's thought, reaction and their purchase intention")
    with gr.Row():
        product_text = gr.Textbox(label="Enter product description")
    with gr.Row():
        text_output_persona = gr.Textbox(label="Customer Personas (CoT 1)")
        text_output_review = gr.Textbox(label="Customer Review (CoT 2)")
        text_output_critics = gr.Textbox(label="Customer  (CoT 3)")
        text_output_verdict = gr.Textbox(label="Customer final verdict")

    btn = gr.Button("Submit")
    btn.click(fn=customer_behavior, inputs=product_text, outputs=[text_output_persona, text_output_review, text_output_critics, text_output_verdict])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eaaca5dac8f00e9118.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
